In [23]:
!pip install transformers


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from tabulate import tabulate
from tqdm import trange
import random

In [25]:
GEN_DF = pd.read_csv(r"/Users/Gacil/OneDrive/Dokumenter/CogSciKA1/NLP/Exam/SDcode/DATAsarcasm_v2/sarcasm_v2/GEN-sarc-notsarc.csv")
GEN_DF.head()
HYP_DF = pd.read_csv(r"/Users/Gacil/OneDrive/Dokumenter/CogSciKA1/NLP/Exam/SDcode/DATAsarcasm_v2/sarcasm_v2/HYP-sarc-notsarc.csv")

RQ_DF = pd.read_csv(r"/Users/Gacil/OneDrive/Dokumenter/CogSciKA1/NLP/Exam/SDcode/DATAsarcasm_v2/sarcasm_v2/RQ-sarc-notsarc.csv")


In [26]:
# Turn 'sarc' = 1 and 'notsarc' = 0
GEN_DF['class'] = GEN_DF['class'].map({'sarc': 1, 'notsarc': 0})
GEN_DF.head()

HYP_DF['class'] = HYP_DF['class'].map({'sarc': 1, 'notsarc': 0})
HYP_DF.head()

RQ_DF['class'] = RQ_DF['class'].map({'sarc': 1 , 'notsarc': 0})
RQ_DF.head()

,class,id,text
0,0,1,"Archie, the ONLY issue that gays don't have a ..."
1,0,2,"No, not really. All that is different is the n..."
2,0,3,It's ashame that everyone keeps looking for th...
3,0,4,"Almost? Usually, that is true, and it involves..."
4,0,5,And so have animals. Plants have been wiped ou...


In [27]:
# Change column name from class to label
GEN_DF.columns = ['label', 'id', 'text']
HYP_DF.columns = ['label', 'id', 'text']
RQ_DF.columns = ['label', 'id', 'text']


In [28]:
# Make a separate dataframe with all the datatypes combined

ALL_DF = pd.concat([GEN_DF, HYP_DF, RQ_DF])

ALL_DF.head()

,label,id,text
0,0,1,"If that's true, then Freedom of Speech is doom..."
1,0,2,Neener neener - is it time to go in from the p...
2,0,3,"Just like the plastic gun fear, the armour pie..."
3,0,4,So geology is a religion because we weren't he...
4,0,5,Well done Monty. Mark that up as your first ev...


In [29]:
# Shuffle
from sklearn.utils import shuffle

GEN_DF = shuffle(GEN_DF, random_state = 1)
HYP_DF = shuffle(HYP_DF, random_state = 1)
RQ_DF = shuffle(RQ_DF, random_state = 1)
ALL_DF = shuffle(ALL_DF, random_state = 1)


In [30]:
# Download BERT tokenizer

tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case = True
    )

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

c:\Users\Gacil\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Gacil\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [31]:
### This code prints a random sentence from the dataset in a table with
### the tokens and token IDs, so you can see how it looks.

# def print_rand_sentence():
#   '''Displays the tokens and respective IDs of a random text sample'''
#   index = random.randint(0, len(GENtext)-1)
#   table = np.array([tokenizer.tokenize(GENtext[index]), 
#                     tokenizer.convert_tokens_to_ids(tokenizer.tokenize(GENtext[index]))]).T
#   print(tabulate(table,
#                  headers = ['Tokens', 'Token IDs'],
#                  tablefmt = 'fancy_grid'))

# print_rand_sentence()

In [32]:
# # Count maximum number of words in text column in each dataframe 
# ## answer = 150

# max(GEN_DF["text"].apply(lambda n: len(n.split())))
# max(HYP_DF["text"].apply(lambda n: len(n.split())))
# max(RQ_DF["text"].apply(lambda n: len(n.split())))

In [33]:
# Specify ratios for training, validation and testin
TV_ratio = 0.2
val_ratio = 0.5 # Thus the test ratio is 0.5


In [34]:
# Split GEN
train_G, TV_G = train_test_split(
    GEN_DF,
    test_size = TV_ratio,
    shuffle = False)

test_G, val_G = train_test_split(
    TV_G,
    test_size = val_ratio,
    shuffle = False)

# Split HYP
train_H, TV_H = train_test_split(
    HYP_DF,
    test_size = TV_ratio,
    shuffle = False)

test_H, val_H = train_test_split(
    TV_H,
    test_size = val_ratio,
    shuffle = False)

# Split RQ
train_R, TV_R = train_test_split(
    RQ_DF,
    test_size = TV_ratio,
    shuffle = False)

test_R, val_R = train_test_split(
    TV_R,
    test_size = val_ratio,
    shuffle = False)


# Split ALL
train_ALL, TV_ALL = train_test_split(
    ALL_DF,
    test_size = TV_ratio,
    shuffle = False)

test_ALL, val_ALL = train_test_split(
    TV_ALL,
    test_size = val_ratio,
    shuffle = False)


In [35]:
# Get text and label values of TRAIN data

GENtext_tr = train_G.text.values
GENlabel_tr = train_G.label.values

HYPtext_tr = train_H.text.values
HYPlabel_tr = train_H.label.values

RQtext_tr = train_R.text.values
RQlabel_tr = train_R.label.values

ALLtext_tr = train_ALL.text.values
ALLlabel_tr = train_ALL.label.values

In [36]:
# Get text and label values of VAL data

GENtext_val = val_G.text.values
GENlabel_val = val_G.label.values

HYPtext_val = val_H.text.values
HYPlabel_val = val_H.label.values

RQtext_val = val_R.text.values
RQlabel_val = val_R.label.values

ALLtext_val = val_ALL.text.values
ALLlabel_val = val_ALL.label.values

In [37]:
# Get text and label values of TEST data

GENtext_test = test_G.text.values
GENlabel_test = test_G.label.values

HYPtext_test = test_H.text.values
HYPlabel_test = test_H.label.values

RQtext_test = test_R.text.values
RQlabel_test = test_R.label.values

ALLtext_test = test_ALL.text.values
ALLlabel_test = test_ALL.label.values

In [38]:
### Pre-processing for BERT

train_id_G = []
train_masks_G = []
val_id_G = []
val_masks_G = []
test_id_G = []
test_masks_G = []

train_id_H = []
train_masks_H = []
val_id_H = []
val_masks_H = []
test_id_H = []
test_masks_H = []

train_id_R = []
train_masks_R = []
val_id_R = []
val_masks_R = []
test_id_R = []
test_masks_R = []

train_id_ALL = []
train_masks_ALL = []
val_id_ALL = []
val_masks_ALL = []
test_id_ALL = []
test_masks_ALL = []


# Define preprocessing code
def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 150,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )




# Loop preprocessing function over GEN train text data
for sample in GENtext_tr:
  encoding_dict = preprocessing(sample, tokenizer)
  train_id_G.append(encoding_dict['input_ids']) 
  train_masks_G.append(encoding_dict['attention_mask'])

train_id_G = torch.cat(train_id_G, dim = 0)
train_masks_G = torch.cat(train_masks_G, dim = 0)
# GEN train label data
labelsGEN_tr = torch.tensor(GENlabel_tr)



# Loop preprocessing function over GEN val text data
for sample in GENtext_val:
  encoding_dict = preprocessing(sample, tokenizer)
  val_id_G.append(encoding_dict['input_ids']) 
  val_masks_G.append(encoding_dict['attention_mask'])

val_id_G = torch.cat(val_id_G, dim = 0)
val_masks_G = torch.cat(val_masks_G, dim = 0)
# GEN val label data
labelsGEN_val = torch.tensor(GENlabel_val)



# Loop preprocessing function over GEN test text data
for sample in GENtext_test:
  encoding_dict = preprocessing(sample, tokenizer)
  test_id_G.append(encoding_dict['input_ids']) 
  test_masks_G.append(encoding_dict['attention_mask'])

test_id_G = torch.cat(test_id_G, dim = 0)
test_masks_G = torch.cat(test_masks_G, dim = 0)
# GEN test label data
labelsGEN_test = torch.tensor(GENlabel_test)








# Loop preprocessing function over HYP train text data
for sample in HYPtext_tr:
  encoding_dict = preprocessing(sample, tokenizer)
  train_id_H.append(encoding_dict['input_ids']) 
  train_masks_H.append(encoding_dict['attention_mask'])

train_id_H = torch.cat(train_id_H, dim = 0)
train_masks_H = torch.cat(train_masks_H, dim = 0)
# HYP train label data
labelsHYP_tr = torch.tensor(HYPlabel_tr)


# Loop preprocessing function over HYP val text data
for sample in HYPtext_val:
  encoding_dict = preprocessing(sample, tokenizer)
  val_id_H.append(encoding_dict['input_ids']) 
  val_masks_H.append(encoding_dict['attention_mask'])

val_id_H = torch.cat(val_id_H, dim = 0)
val_masks_H = torch.cat(val_masks_H, dim = 0)
# HYP val label data
labelsHYP_val = torch.tensor(HYPlabel_val)


# Loop preprocessing function over HYP test text data
for sample in HYPtext_test:
  encoding_dict = preprocessing(sample, tokenizer)
  test_id_H.append(encoding_dict['input_ids']) 
  test_masks_H.append(encoding_dict['attention_mask'])

test_id_H = torch.cat(test_id_H, dim = 0)
test_masks_H = torch.cat(test_masks_H, dim = 0)
# HYP test label data
labelsHYP_test = torch.tensor(HYPlabel_test)








# Loop preprocessing function over RQ train text data
for sample in RQtext_tr:
  encoding_dict = preprocessing(sample, tokenizer)
  train_id_R.append(encoding_dict['input_ids']) 
  train_masks_R.append(encoding_dict['attention_mask'])

train_id_R = torch.cat(train_id_R, dim = 0)
train_masks_R = torch.cat(train_masks_R, dim = 0)
# RQ train label data
labelsRQ_tr = torch.tensor(RQlabel_tr)



# Loop preprocessing function over RQ val text data
for sample in RQtext_val:
  encoding_dict = preprocessing(sample, tokenizer)
  val_id_R.append(encoding_dict['input_ids']) 
  val_masks_R.append(encoding_dict['attention_mask'])

val_id_R = torch.cat(val_id_R, dim = 0)
val_masks_R = torch.cat(val_masks_R, dim = 0)
# RQ val label data
labelsRQ_val = torch.tensor(RQlabel_val)



# Loop preprocessing function over RQ test text data
for sample in RQtext_test:
  encoding_dict = preprocessing(sample, tokenizer)
  test_id_R.append(encoding_dict['input_ids']) 
  test_masks_R.append(encoding_dict['attention_mask'])

test_id_R = torch.cat(test_id_R, dim = 0)
test_masks_R = torch.cat(test_masks_R, dim = 0)
# RQ test label data
labelsRQ_test = torch.tensor(RQlabel_test)








# Loop preprocessing function over ALL train text data
for sample in ALLtext_tr:
  encoding_dict = preprocessing(sample, tokenizer)
  train_id_ALL.append(encoding_dict['input_ids']) 
  train_masks_ALL.append(encoding_dict['attention_mask'])

train_id_ALL = torch.cat(train_id_ALL, dim = 0)
train_masks_ALL = torch.cat(train_masks_ALL, dim = 0)
# RQ train label data
labelsALL_tr = torch.tensor(ALLlabel_tr)



# Loop preprocessing function over RQ val text data
for sample in ALLtext_val:
  encoding_dict = preprocessing(sample, tokenizer)
  val_id_ALL.append(encoding_dict['input_ids']) 
  val_masks_ALL.append(encoding_dict['attention_mask'])

val_id_ALL = torch.cat(val_id_ALL, dim = 0)
val_masks_ALL = torch.cat(val_masks_ALL, dim = 0)
# RQ val label data
labelsALL_val = torch.tensor(ALLlabel_val)



# Loop preprocessing function over RQ test text data
for sample in ALLtext_test:
  encoding_dict = preprocessing(sample, tokenizer)
  test_id_ALL.append(encoding_dict['input_ids']) 
  test_masks_ALL.append(encoding_dict['attention_mask'])

test_id_ALL = torch.cat(test_id_ALL, dim = 0)
test_masks_ALL = torch.cat(test_masks_ALL, dim = 0)
# RQ test label data
labelsALL_test = torch.tensor(ALLlabel_test)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\Gacil\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [39]:
# Checking

labelsGEN_tr[32]

tensor(1)

In [40]:
# Turn train_idx and val_idx into numpy arrays




In [41]:

batch_size = 20

# GEN

# Train and validation sets
train_setG = TensorDataset(train_id_G, 
                          train_masks_G, 
                          labelsGEN_tr)

val_setG = TensorDataset(val_id_G, 
                        val_masks_G, 
                        labelsGEN_val)



# Prepare DataLoader
train_dataloaderG = DataLoader(
            train_setG,
            sampler = RandomSampler(train_setG),
            batch_size = batch_size
        )

validation_dataloaderG = DataLoader(
            val_setG,
            sampler = SequentialSampler(val_setG),
            batch_size = batch_size
        )

In [42]:
# HYP

# Train and validation sets
train_setH = TensorDataset(train_id_H, 
                          train_masks_H, 
                          labelsHYP_tr)

val_setH = TensorDataset(val_id_H, 
                        val_masks_H, 
                        labelsHYP_val)



# Prepare DataLoader
train_dataloaderH = DataLoader(
            train_setH,
            sampler = RandomSampler(train_setH),
            batch_size = batch_size
        )

validation_dataloaderH = DataLoader(
            val_setH,
            sampler = SequentialSampler(val_setH),
            batch_size = batch_size
        )

In [43]:
# RQ

# Train and validation sets
train_setR = TensorDataset(train_id_R, 
                          train_masks_R, 
                          labelsRQ_tr)

val_setR = TensorDataset(val_id_R, 
                        val_masks_R, 
                        labelsRQ_val)



# Prepare DataLoader
train_dataloaderR = DataLoader(
            train_setR,
            sampler = RandomSampler(train_setR),
            batch_size = batch_size
        )

validation_dataloaderR = DataLoader(
            val_setR,
            sampler = SequentialSampler(val_setR),
            batch_size = batch_size
        )

In [44]:
# ALL

# Train and validation sets
train_setALL = TensorDataset(train_id_ALL, 
                          train_masks_ALL, 
                          labelsALL_tr)

val_setALL = TensorDataset(val_id_ALL, 
                        val_masks_ALL, 
                        labelsALL_val)



# Prepare DataLoader
train_dataloaderALL = DataLoader(
            train_setALL,
            sampler = RandomSampler(train_setALL),
            batch_size = batch_size
        )

validation_dataloaderALL = DataLoader(
            val_setALL,
            sampler = SequentialSampler(val_setALL),
            batch_size = batch_size
        )

Create functions for assessing validation metrics (accuracy, precision, recall, specificity)


In [45]:

def b_tp(preds, labels):
  '''Returns True Positives (TP): count of correct predictions of actual class 1'''
  return sum([preds == labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_fp(preds, labels):
  '''Returns False Positives (FP): count of wrong predictions of actual class 1'''
  return sum([preds != labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_tn(preds, labels):
  '''Returns True Negatives (TN): count of correct predictions of actual class 0'''
  return sum([preds == labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_fn(preds, labels):
  '''Returns False Negatives (FN): count of wrong predictions of actual class 0'''
  return sum([preds != labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_metrics(preds, labels):
  '''
  Returns the following metrics:
    - accuracy    = (TP + TN) / N
    - precision   = TP / (TP + FP)
    - recall      = TP / (TP + FN)
    - specificity = TN / (TN + FP)
  '''
  preds = np.argmax(preds, axis = 1).flatten()
  labels = labels.flatten()
  tp = b_tp(preds, labels)
  tn = b_tn(preds, labels)
  fp = b_fp(preds, labels)
  fn = b_fn(preds, labels)
  b_accuracy = (tp + tn) / len(labels)
  b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'
  b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'
  b_specificity = tn / (tn + fp) if (tn + fp) > 0 else 'nan'
  return b_accuracy, b_precision, b_recall, b_specificity

Load BERT for sequence classification model

In [46]:
# Load the BertForSequenceClassification model1 for GEN 
model1 = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5. See: https://arxiv.org/pdf/1810.04805.pdf
optimizer1 = torch.optim.AdamW(model1.parameters(), 
                              lr = 1e-5,
                              eps = 1e-08
                              )




# Load the BertForSequenceClassification model2 for HYP
model2 = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5. See: https://arxiv.org/pdf/1810.04805.pdf
optimizer2 = torch.optim.AdamW(model2.parameters(), 
                              lr = 1e-5,
                              eps = 1e-08
                              )



# Load the BertForSequenceClassification model3 for RQ
model3 = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5. See: https://arxiv.org/pdf/1810.04805.pdf
optimizer3 = torch.optim.AdamW(model3.parameters(), 
                              lr = 1e-5,
                              eps = 1e-08
                              )



# Load the BertForSequenceClassification model4 for ALL DATA
model4 = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5. See: https://arxiv.org/pdf/1810.04805.pdf
optimizer4 = torch.optim.AdamW(model4.parameters(), 
                              lr = 1e-5,
                              eps = 1e-08
                              )



Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [47]:
 torch.cuda.empty_cache()

We start with training GEN

In [50]:
model1.cuda()

device = torch.device('cuda')

# Recommended number of epochs: 2, 3, 4. See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 4
print("[INFO:] Training classifier on GEN data...")

for _ in trange(epochs, desc = 'Epoch'):
    
    # ========== Training ==========
    
    # Set model to training mode
    model1.train()
    
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloaderG):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer1.zero_grad()
        # Forward pass
        train_output_GEN = model1(b_input_ids, 
                             token_type_ids = None, 
                             attention_mask = b_input_mask, 
                             labels = b_labels)
        # Backward pass
        train_output_GEN.loss.backward()
        optimizer1.step()
        # Update tracking variables
        tr_loss += train_output_GEN.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

# ========== Validation ==========

    # Set model to evaluation mode
    model1.eval()

    # Tracking variables 
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_specificity = []

    for batch in validation_dataloaderG:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
          eval_output_GEN = model1(b_input_ids, 
                              token_type_ids = None, 
                              attention_mask = b_input_mask)
        logits_GEN = eval_output_GEN.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate validation metrics
        b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits_GEN, label_ids)
        val_accuracy.append(b_accuracy)
        # Update precision only when (tp + fp) !=0; ignore nan
        if b_precision != 'nan': val_precision.append(b_precision)
        # Update recall only when (tp + fn) !=0; ignore nan
        if b_recall != 'nan': val_recall.append(b_recall)
        # Update specificity only when (tn + fp) !=0; ignore nan
        if b_specificity != 'nan': val_specificity.append(b_specificity)

    print('\n\t - GEN Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - GEN Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - GEN Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
    print('\t - GEN Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
    print('\t - GEN Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '\t - Validation Specificity: NaN')


AssertionError: Torch not compiled with CUDA enabled

In [33]:
# Save the entire model to a file (model1)
torch.save({'model_state_dict': model1.state_dict(),
            'optimizer_state_dict': optimizer1.state_dict()}, 'model1.pth')


In [27]:
# Load model1

model1 = torch.load(('model1.pth'),model1.load_state_dict(model1['model_state_dict']),
optimizer1.load_state_dict(model1['optimizer1_state_dict']))

NameError: ignored

This model can be loaded again by writing:

"
import torch

# Load the model and optimizer's state
model1 = torch.load('model1.pth')
model.load_state_dict(model1['model_state_dict'])
optimizer.load_state_dict(model1['optimizer1_state_dict'])

# Use the model and optimizer to continue training or make predictions
"

Now train HYP

In [34]:
model2.cuda() 

print("[INFO:] Training classifier on HYP data...")

for _ in trange(epochs, desc = 'Epoch'):
    
    # ========== Training ==========
    
    # Set model to training mode
    model2.train()
    
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloaderH):
        batch = tuple(t.to(device) for t in batch)
        c_input_ids, c_input_mask, c_labels = batch
        optimizer2.zero_grad()
        # Forward pass
        train_output_HYP = model2(c_input_ids, 
                             token_type_ids = None, 
                             attention_mask = c_input_mask, 
                             labels = c_labels)
        # Backward pass
        train_output_HYP.loss.backward()
        optimizer2.step()
        # Update tracking variables
        tr_loss += train_output_HYP.loss.item()
        nb_tr_examples += c_input_ids.size(0)
        nb_tr_steps += 1

# ========== Validation ==========

    # Set model to evaluation mode
    model2.eval()

    # Tracking variables 
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_specificity = []

    for batch in validation_dataloaderH:
        batch = tuple(t.to(device) for t in batch)
        c_input_ids, c_input_mask, c_labels = batch
        with torch.no_grad():
          # Forward pass
          eval_output_HYP = model2(c_input_ids, 
                              token_type_ids = None, 
                              attention_mask = c_input_mask)
        logits_HYP = eval_output_HYP.logits.detach().cpu().numpy()
        label_ids = c_labels.to('cpu').numpy()
        # Calculate validation metrics
        b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits_HYP, label_ids)
        val_accuracy.append(b_accuracy)
        # Update precision only when (tp + fp) !=0; ignore nan
        if b_precision != 'nan': val_precision.append(b_precision)
        # Update recall only when (tp + fn) !=0; ignore nan
        if b_recall != 'nan': val_recall.append(b_recall)
        # Update specificity only when (tn + fp) !=0; ignore nan
        if b_specificity != 'nan': val_specificity.append(b_specificity)

    print('\n\t - HYP Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - HYP Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - HYP Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
    print('\t - HYP Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
    print('\t - HYP Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '\t - Validation Specificity: NaN')


[INFO:] Training classifier on HYP data...


Epoch:  25%|██▌       | 1/4 [00:25<01:17, 25.82s/it]


	 - HYP Train loss: 0.6878
	 - HYP Validation Accuracy: 0.5951
	 - HYP Validation Precision: 0.6890
	 - HYP Validation Recall: 0.5818
	 - HYP Validation Specificity: 0.6095



Epoch:  50%|█████     | 2/4 [00:51<00:51, 25.93s/it]


	 - HYP Train loss: 0.6166
	 - HYP Validation Accuracy: 0.7025
	 - HYP Validation Precision: 0.8049
	 - HYP Validation Recall: 0.6691
	 - HYP Validation Specificity: 0.7520



Epoch:  75%|███████▌  | 3/4 [01:17<00:25, 25.75s/it]


	 - HYP Train loss: 0.4998
	 - HYP Validation Accuracy: 0.6593
	 - HYP Validation Precision: 0.7564
	 - HYP Validation Recall: 0.6272
	 - HYP Validation Specificity: 0.6981



Epoch: 100%|██████████| 4/4 [01:43<00:00, 25.77s/it]


	 - HYP Train loss: 0.3512
	 - HYP Validation Accuracy: 0.7554
	 - HYP Validation Precision: 0.8304
	 - HYP Validation Recall: 0.7461
	 - HYP Validation Specificity: 0.7705



In [35]:
# Save the entire model to a file (model1)
torch.save({'model_state_dict': model2.state_dict(),
            'optimizer_state_dict': optimizer2.state_dict()}, 'model2.pth')

Now train RQ

In [36]:
model3.cuda()


print("[INFO:] Training classifier on RQ data...")

for _ in trange(epochs, desc = 'Epoch'):
    
    # ========== Training ==========
    
    # Set model to training mode
    model3.train()
    
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloaderR):
        batch = tuple(t.to(device) for t in batch)
        d_input_ids, d_input_mask, d_labels = batch
        optimizer3.zero_grad()
        # Forward pass
        train_output_RQ = model3(d_input_ids, 
                             token_type_ids = None, 
                             attention_mask = d_input_mask, 
                             labels = d_labels)
        # Backward pass
        train_output_RQ.loss.backward()
        optimizer3.step()
        # Update tracking variables
        tr_loss += train_output_RQ.loss.item()
        nb_tr_examples += d_input_ids.size(0)
        nb_tr_steps += 1

# ========== Validation ==========

    # Set model to evaluation mode
    model3.eval()

    # Tracking variables 
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_specificity = []

    for batch in validation_dataloaderR:
        batch = tuple(t.to(device) for t in batch)
        d_input_ids, d_input_mask, d_labels = batch
        with torch.no_grad():
          # Forward pass
          eval_output_RQ = model3(d_input_ids, 
                              token_type_ids = None, 
                              attention_mask = d_input_mask)
        logits_RQ = eval_output_RQ.logits.detach().cpu().numpy()
        label_ids = d_labels.to('cpu').numpy()
        # Calculate validation metrics
        b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits_RQ, label_ids)
        val_accuracy.append(b_accuracy)
        # Update precision only when (tp + fp) !=0; ignore nan
        if b_precision != 'nan': val_precision.append(b_precision)
        # Update recall only when (tp + fn) !=0; ignore nan
        if b_recall != 'nan': val_recall.append(b_recall)
        # Update specificity only when (tn + fp) !=0; ignore nan
        if b_specificity != 'nan': val_specificity.append(b_specificity)

    print('\n\t - RQ Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - RQ Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - RQ Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
    print('\t - RQ Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
    print('\t - RQ Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '\t - Validation Specificity: NaN')


[INFO:] Training classifier on RQ data...


Epoch:  25%|██▌       | 1/4 [00:38<01:54, 38.27s/it]


	 - RQ Train loss: 0.6133
	 - RQ Validation Accuracy: 0.7919
	 - RQ Validation Precision: 0.8457
	 - RQ Validation Recall: 0.7101
	 - RQ Validation Specificity: 0.8511



Epoch:  50%|█████     | 2/4 [01:16<01:16, 38.03s/it]


	 - RQ Train loss: 0.4503
	 - RQ Validation Accuracy: 0.7732
	 - RQ Validation Precision: 0.7156
	 - RQ Validation Recall: 0.8673
	 - RQ Validation Specificity: 0.6597



Epoch:  75%|███████▌  | 3/4 [01:54<00:38, 38.06s/it]


	 - RQ Train loss: 0.3473
	 - RQ Validation Accuracy: 0.7909
	 - RQ Validation Precision: 0.7871
	 - RQ Validation Recall: 0.7899
	 - RQ Validation Specificity: 0.7687



Epoch: 100%|██████████| 4/4 [02:32<00:00, 38.06s/it]


	 - RQ Train loss: 0.2462
	 - RQ Validation Accuracy: 0.7687
	 - RQ Validation Precision: 0.7189
	 - RQ Validation Recall: 0.8284
	 - RQ Validation Specificity: 0.6685



In [37]:
# Save the entire model to a file (model1)
torch.save({'model_state_dict': model3.state_dict(),
            'optimizer_state_dict': optimizer3.state_dict()}, 'model3.pth')

Train model on ALL DATA

In [38]:
model4.cuda()

print("[INFO:] Training classifier on ALL data...")

for _ in trange(epochs, desc = 'Epoch'):
    
    # ========== Training ==========
    
    # Set model to training mode
    model4.train()
    
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloaderR):
        batch = tuple(t.to(device) for t in batch)
        e_input_ids, e_input_mask, e_labels = batch
        optimizer4.zero_grad()
        # Forward pass
        train_output_ALL = model4(e_input_ids, 
                             token_type_ids = None, 
                             attention_mask = e_input_mask, 
                             labels = e_labels)
        # Backward pass
        train_output_ALL.loss.backward()
        optimizer4.step()
        # Update tracking variables
        tr_loss += train_output_ALL.loss.item()
        nb_tr_examples += e_input_ids.size(0)
        nb_tr_steps += 1

# ========== Validation ==========

    # Set model to evaluation mode
    model4.eval()

    # Tracking variables 
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_specificity = []

    for batch in validation_dataloaderR:
        batch = tuple(t.to(device) for t in batch)
        e_input_ids, e_input_mask, e_labels = batch
        with torch.no_grad():
          # Forward pass
          eval_output_ALL = model4(e_input_ids, 
                              token_type_ids = None, 
                              attention_mask = e_input_mask)
        logits_ALL = eval_output_ALL.logits.detach().cpu().numpy()
        label_ids = e_labels.to('cpu').numpy()
        # Calculate validation metrics
        b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits_ALL, label_ids)
        val_accuracy.append(b_accuracy)
        # Update precision only when (tp + fp) !=0; ignore nan
        if b_precision != 'nan': val_precision.append(b_precision)
        # Update recall only when (tp + fn) !=0; ignore nan
        if b_recall != 'nan': val_recall.append(b_recall)
        # Update specificity only when (tn + fp) !=0; ignore nan
        if b_specificity != 'nan': val_specificity.append(b_specificity)

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
    print('\t - Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '\t - Validation Specificity: NaN')


[INFO:] Training classifier on ALL data...


Epoch:  25%|██▌       | 1/4 [00:38<01:54, 38.18s/it]


	 - Train loss: 0.6067
	 - Validation Accuracy: 0.7707
	 - Validation Precision: 0.8012
	 - Validation Recall: 0.7361
	 - Validation Specificity: 0.7942



Epoch:  50%|█████     | 2/4 [01:16<01:16, 38.00s/it]


	 - Train loss: 0.4659
	 - Validation Accuracy: 0.7641
	 - Validation Precision: 0.8306
	 - Validation Recall: 0.6740
	 - Validation Specificity: 0.8236



Epoch:  75%|███████▌  | 3/4 [01:54<00:38, 38.00s/it]


	 - Train loss: 0.3704
	 - Validation Accuracy: 0.7854
	 - Validation Precision: 0.8015
	 - Validation Recall: 0.7553
	 - Validation Specificity: 0.7849



Epoch: 100%|██████████| 4/4 [02:32<00:00, 38.04s/it]


	 - Train loss: 0.2650
	 - Validation Accuracy: 0.7631
	 - Validation Precision: 0.7164
	 - Validation Recall: 0.8610
	 - Validation Specificity: 0.6344



In [39]:
# Save the entire model to a file (model1)
torch.save({'model_state_dict': model4.state_dict(),
            'optimizer_state_dict': optimizer4.state_dict()}, 'model4.pth')

Create plot of loss history if this is needed


In [40]:
# def create_plot(tr_loss):
#     fig, ax = plt.subplots()
#     ax.plot(loss_history, label = "training loss")
#     plt.legend()
#     plt.show()
#     #plot_outpath = os.path.join("loss.png")
#     #fig.savefig(plot_outpath) 
#     return 

# create_plot(tr_loss)

#epoch1 = (0.5135, 0.8740, 0.8638, 0.8912, 0.8516)
#epoch2 = 

Now I test the model ?

In [41]:
# Test set
test_setG = TensorDataset(test_id_G, 
                        test_masks_G, 
                        labelsGEN_test)

# Prepare dataloader

test_dataloaderG = DataLoader(
            test_setG,
            sampler = SequentialSampler(test_setG),
            batch_size = batch_size
        )



In [42]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [43]:
# # Set the model to evaluation mode
# model1.eval()

# # Initialize the list to store the predictions
# predictionsGEN = []

#  # Use the model to make a prediction
# with torch.no_grad():
#         output = model1(test_id_G, test_masks_G)
#         _, prediction = torch.max(output, dim=1)

#     # Add the prediction to the list
# predictionsGEN.append(prediction.item())

# # Calculate the accuracy of the predictions
# accuracy = sum(predictionsGEN == labelsGEN_test) / len(labelsGEN_test)
# print(f'Accuracy: {accuracy:.2f}')

RuntimeError: ignored

In [44]:
# predictions = []

# with torch.no_grad():
#   output = model1(test_id_G.to(device), token_type_ids = None, attention_mask = test_masks_G.to(device)),
#   prediction = 'Sarcastic' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'Not sarcastic'
#   predictions.append(prediction.item())

# print('Predicted Class: ', prediction) 

# accuracy = sum(predictions == labelsGEN_test) / len(labelsGEN_test)
# print(f'Accuracy: {accuracy:.2f}')


AttributeError: ignored

In [ ]:
# # Plot
# predicted = model1(X_vect).detach().numpy()
# print(classification_report(y, 
#                             np.where(predicted > 0.5, 1, 0),
#                             target_names = ["Negative", "Positive"]))

In [ ]:
# pred = model1.predict(test_id_G)